In [32]:
# установка необх.библиотек
# import sys
!pip3 install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

!pip3 install pandas
!pip3 install xlrd
!pip3 install scipy
!pip3 install statsmodels
!pip3 install matplotlib
!pip3 install seaborn

Requirement already up-to-date: google-api-python-client in /Users/nsh/.pyenv/versions/3.7.2/lib/python3.7/site-packages (1.12.3)
Requirement already up-to-date: google-auth-httplib2 in /Users/nsh/.pyenv/versions/3.7.2/lib/python3.7/site-packages (0.0.4)
Requirement already up-to-date: google-auth-oauthlib in /Users/nsh/.pyenv/versions/3.7.2/lib/python3.7/site-packages (0.4.1)
     |████████████████████████████████| 103 kB 581 kB/s eta 0:00:01


In [8]:
# импорты 
import io
from googleapiclient.http import MediaIoBaseDownload
import googleapiclient.discovery
from apiclient import discovery
from httplib2 import Http
import oauth2client
from oauth2client import file, client, tools

import pandas as pd
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

In [59]:
# немного настроек для графиков
%matplotlib inline

# Some plot styling preferences
plt.style.use('seaborn-whitegrid')
font = {'family' : 'Helvetica',
        'weight' : 'bold',
        'size'   : 14}                                              

In [3]:
# проводим аутентификацию через гугловую апишку

obj = lambda: None
lmao = {"auth_host_name":'localhost', 'noauth_local_webserver':'store_true', 'auth_host_port':[8080, 8090], 'logging_level':'ERROR'}
for k, v in lmao.items():
    setattr(obj, k, v)
    
# authorization boilerplate code
SCOPES = 'https://www.googleapis.com/auth/drive.readonly'
store = file.Storage('token.json')
creds = store.get()
# The following will give you a link if token.json does not exist, the link allows the user to give this app permission
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('credential/credentials.json', SCOPES)
    creds = tools.run_flow(flow, store, obj)

/Users/nsh/.pyenv/versions/3.7.2/lib/python3.7/site-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access token.json: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))



Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=783548059285-jduaap6ga9va97jpeit08lfaohlq1bvv.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&access_type=offline&response_type=code

Enter verification code: 4/1AfDhmrimGjVvJ8VDSny1Fyb5Jj2r-iHyT588-DlTIU43YtXEn4Sf7soXwQs
Authentication successful.


In [4]:
# путь к исходному файлу с результатами теста
path_to_file = './data/split_form.xlsx'

In [87]:
# считываем необходимый файл. Удобно забирать прямо с диска - все изменения на диске тянем сразу 
#  (перезаписываем файл на локальной машине)

DRIVE = googleapiclient.discovery.build(serviceName='drive'
                                        , version='v3'
                                        , http=creds.authorize(Http()))
# if you get the shareable link, the link contains this id, replace the file_id below

file_id = '16-kZKQIdjeKErkEyPilollVPCj2CHzy1'
request = DRIVE.files().get_media(fileId=file_id)
# replace the filename and extension in the first field below
fh = io.FileIO(path_to_file, mode='w')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100)) 

Download 100%.


# Формулировка гипотезы

    Если убрать поле для ввода Name on card, то вырастет конверсия в нажатие кнопки Pay, т.к. пользователям придётся вводит меньше данных.


    P.S. Необходимо чертить "линию на песке" - ожидаемый % роста конверсии. И изначальную конверсию, т.к. даже незначительный рост можно интерпрепировать у свою сторону. 
### Костылем попробуем сформулировать гипотезу, считав данные с исходного df. Безусловно, нехорошо, т.к. гипотезы должны быть сформулированные без подглядываний.  

In [6]:
# чтение файла в датафрейм
split = pd.read_excel(path_to_file)

split.sample(3)

,DEVICES_BRANDS,BROWSERS,DEVICE_TYPE,Conversion,A/B,config_os,config_os_version,config_resolution,idvisit,platform_v4
7617,SAMSUNG,CHROME MOBILE,SMARTPHONE,0,new,AND,7,360x640,5127365,MOB
9061,Other,CHROME MOBILE,SMARTPHONE,0,old,AND,7,360x720,4391393,MOB
23515,Other,CHROME MOBILE,SMARTPHONE,0,new,AND,4.4,360x640,6022098,MOB


In [88]:
# список измерений для группировки и вычислений
diminsion_col = ['DEVICES_BRANDS', 'BROWSERS', 'DEVICE_TYPE', 'Conversion'
                 , 'A/B', 'config_os', 'config_os_version', 'config_resolution', 'platform_v4']

In [89]:
# No action required: Leave the original dataframe untouched and sort the new dataframe
split2 = split.copy()

In [90]:
# считаем кол-во визитов в разрезе необходимых измерений
split2['visit_cnt'] = split2.groupby(diminsion_col)['idvisit'].transform('count')


split2 = split2.astype({"visit_cnt": int}, errors='ignore')

split2 = split2.filter(['DEVICES_BRANDS', 'BROWSERS', 'DEVICE_TYPE', 'Conversion'
                 , 'A/B', 'config_os', 'config_resolution', 'config_os_version', 'platform_v4', 'visit_cnt'])

split2.sample(3)

,DEVICES_BRANDS,BROWSERS,DEVICE_TYPE,Conversion,A/B,config_os,config_resolution,config_os_version,platform_v4,visit_cnt
7591,Other,CHROME MOBILE,SMARTPHONE,0,new,AND,360x640,7,MOB,2355.0
16345,Other,CHROME MOBILE,SMARTPHONE,0,new,AND,424x753,7,MOB,49.0
18527,Other,CHROME,DESKTOP,0,old,WIN,1280x1024,7,WEB,18.0


In [91]:
old = split2[split2['A/B']=='old']
table = pd.pivot_table(old, values=["visit_cnt"],
                       index=["A/B"], columns=["Conversion"], 
                       aggfunc=len, margins=True, 
                       dropna=True, fill_value=0)

table2 = table.div(table.iloc[:,-1], axis=0)
table, table2

(           visit_cnt               
 Conversion         0     1      All
 A/B                                
 old             9806  1350  11085.0
 All             9739  1346  11085.0,
            visit_cnt               
 Conversion         0         1  All
 A/B                                
 old         0.884619  0.121786  1.0
 All         0.878575  0.121425  1.0)

## Нулевая гипотеза
> ### Если убрать поле для ввода Name on card, то конверсия повыситься с 12,1% до 15%

# 

# Sampling

Поскольку мы не знаем, будет ли новый дизайн работать лучше или хуже (или будет таким же?), Как наш текущий, мы выберем двусторонний тест:
> # Hₒ: p = pₒ
> # Hₐ: p ≠ pₒ
    - где p и pₒ обозначают коэффициент конверсии нового и старого дизайна соответственно. 
    
    Мы также установим уровень достоверности 95%:
> # α = 0,05
    
    Значение α - это порог, который мы устанавливаем, с помощью которого мы говорим: «Если вероятность увидеть результат как экстремальный или более высокий (p-значение) ниже, чем α, то мы отвергаем нулевую гипотезу». Поскольку наш α = 0,05 (что указывает на вероятность 5%), наша уверенность (1 - α) составляет 95%.

## Cколько человек должно быть в каждой группе?

Размер необходимой нам выборки оценивается с помощью так называемого анализа мощности и зависит от нескольких факторов:
### Мощность теста (1 - β) 
    - это вероятность обнаружения статистической разницы между группами в нашем тесте, когда разница действительно присутствует. Обычно по соглашению устанавливается на 0,8 (если вам интересно, вот дополнительная информация о статистической мощности).
### Альфа-значение (α) 
     - Критическое значение, которое мы ранее установили равным 0,05
### Размер эффекта 
    - насколько большой мы ожидаем увидеть разницу между коэффициентами конверсии.

In [92]:
mpl.rc('font', **font)
effect_size = sms.proportion_effectsize(0.121, 0.15)    # Calculating effect size based on our expected rates

required_n = sms.NormalIndPower().solve_power(
    effect_size, 
    power=0.8, 
    alpha=0.05, 
    ratio=1
    )   

required_n = ceil(required_n)                          # Rounding up to next whole number                          

print(f'We’d need at least {required_n} observations for each group')

We’d need at least 2181 observations for each group


   > Установка параметра мощности на 0,8 на практике означает, что если существует реальная разница в коэффициенте конверсии между нашими проектами, предполагая, что разница является той, которую мы оценили (12,1% против 15%), у нас есть 80% шанс определить его как статистически значимый в нашем тесте с размером выборки, который мы рассчитали.

In [93]:
pd.crosstab(split2['A/B'], split2['Conversion']
            , margins=True, margins_name="Total")

Conversion,0,1,Total
A/B,,,
new,13564,1865,15429
old,9806,1350,11156
Total,23370,3215,26585


       При рекомендуемом значение выборки 2181 мы имеем кол-во выше: новая - 15429 и старая - 11156. Закладывался ли изначально такой размер выборки? Также наблюдается диспропорция в сторону новой группы. Необходимо разделять 50/50 или делать тестовую группу меньше, т.к. не знаем как она заафектит.

### Рекомендации

     >  1) Точная формулировка гипотезы. Какой % конверсии сейчас и сколько хотим получить при запуске новой формы.
     >   2) Дизайн теста. Сколько человек достаточно для получения достоверных ожидаемых результатов. От этого получим и время проведения.
     >   3) Необходимо понимать длительность проведения А/В теста и поведение пользователей во времени и получены ли данные в один период - добавить колонку timestamp.
     >   4 ) Отсутствует user_id - хотелось бы проверить по пользователю или Cookies-файлу не было ли повторных транзакций у форме. И не было ли в пользователя пересечений тестовая/контрольная группа.
     > 5)  Перезапуск. Можно копаться в текущих результатах, но похоже на проблему подглядывания. Соответственно, есть риск сделать вывод на нерелевантных данных (диспропорция с размере групп, типах платформ и т.д.), что дороже, чем следовать правильной методологии проведения теста.
     > 6) Добавляю детализацию по микроконверсии сегментов трафика при проведении теста в тетрадку micro_conversion_addit_split_test для понимания общего хода теста и его ценности.

Ссылка на тему проблемы подглядывания и дизайна теста: 
https://gopractice.ru/how-not-to-analyze-abtests/